# External Forcing
External forcing fields (usually) refer to atmospheric paramters that modify the temperature, salinity, or momentum in the upper ocean. However, there are some occasionally-used external forcing fields that are implemented at the ocean floor, such as geothermal heat flux. The following table lists commonly-used external forcing fields:

### Common External Forcing Fields

| Field Name | Model Variable | Units | Notes | 
| ---------- | -------------- | ----- | :---- |
| Zonal Wind | `UVEL`           | m/s   | Wind is positive eastward *in model coordinates*, which is not necessarily east on the real globe, depending on the orientation of the model grid |
| Meridional Wind | `VVEL` | m/s | Wind is positive northward *in model coordinates*, which is not necessarily east on the real globe, depending on the orientation of the model grid |
| Surface Air Temperature | `ATEMP` | K | If Celsius is used, an offset of 273.15 must be applied in the data.exf file. |
| Surface Specific Humidity | `AQH` | kg H$_2$O per kg air | |
| Downward shortwave radiation | `SWDOWN` | W/m$^2$ | Downward positive |
| Downward longwave radiation | `LWDOWN` | W/m$^2$ | Downward positive |
| Precipitation | `PRECIP` | m/s |  |
| Runoff | `RUNOFF` | m/s |  |

## The `exf` package
In MITgcm, the primary way to implement external forcing conditions is through the `exf` package. 

### Compile-time considerations
To enable the `exf` package for compilation, add a line for `exf` in the `packages.conf` files. 

```{note}
While the `exf` package does not strictly require the use of the `cal` package, many of the `exf` funtionality was built around tools from the `cal` package. With this in mind, it's recommended that the `cal` package also be enabled by adding a line for `cal` in the `packages.conf` file.
```

Note that by default, the [EXF_OPTIONS.h](https://github.com/MITgcm/MITgcm/blob/master/pkg/exf/EXF_OPTIONS.h) file is configured to use all of the fields listed in the table above. Further information is provided on the [MITgcm documentation page](https://mitgcm.readthedocs.io/en/latest/phys_pkgs/exf.html) for the `exf` package.

### Run-time considerations

#### data.exf

The data.exf file is a required file when using the external forcing package. There are four main namelists in this file:

| Namelist | Purpose | 
| -------- | :------ |
| `EXF_NML_01` | Set parameters for reading, scaling, and applying external forcing conditions |
| `EXF_NML_02` | Set the file names and start dates for each external field file |
| `EXF_NML_03` | Set scaling parameters for the forcing fields  |
| `EXF_NML_04` | Set the spatial interpolation method for each field |
| `EXF_NML_OBCS` | When using open boundaries, the time stepping is outlined in this namelist. Not required if boundary conditions are not used. |

#### Formatting the fields
The external forcing fields are stored in files sized (time, Ny, Nx). There are two ways the files can be organized:
1. All of the external forcing conditions can be added in a single file, or
2. The external forcing conditions can be stored in annual files. In this case, each file must end in an underscore and the year. For example, `UWIND_2001`, `UWIND_2002`, etc. When using this option, the flag `useExfYearlyFields= .TRUE.,` must be set in `EXF_NML_01` and the file name is specified as `uwindfile='UWIND'`, in `EXF_NML_02` - the underscore and year will be appended based on the model time.

#### Temporal Interpolation
In the data.exf file, temporal information is provided for each file in `EXF_NML_02`. For example, consider the following information for the zonal wind file:

```
 uwindstartdate1   = 19920101,
 uwindstartdate2   = 030000,
 uwindperiod       = 21600.0,
```

This information indicates the first timestep in the file corresponds to the date 1 January 1992 at 3:00 am. The first date line corresponds to the date (YYYYMMDD) and the second to the time (HHMMSS). The final line corresponds to the temporal resolution in the model - in this case, the file provides 6-hourly wind speeds i.e. the second timesteps corresponds to 1 January 1992 at 9:00 am.

In the special case that a climatological file is provided (i.e. if the file is a monthly file that will be repeated each year in the model), then the date information is omitted and the period is set to `uwindperiod = -12,`.

To determine the external forcing value at a given time, the model **linearly interpolates between the two closest timesteps** at each model point.

#### Spatial Interpolation
There are two ways that the model can spatially interpolate from the provided conditions to the model grid.

First, the external forcing conditions can be provided on the exact same grid as the model. In this case, a flag of 0 should be provided in `EXF_NML_04`:

```
 uwind_interpMethod = 0,
```

Second, the model can linearly interpolate the external forcing conditions onto the model grid. In this case, the geographic information for the locations of the external forcing field must be provided in `EXF_NML_04`, e.g.

```
 uwind_lon0        =   0.0000000D0,
 uwind_lat0        = -89.4628220D0,
 uwind_lon_inc     =   0.7031250D0,
 uwind_lat_inc     = 0.6958694,0.6999817,0.7009048,0.7012634,0.7014313,
                     245*0.7017418,
                     0.7014313,0.7012634,0.7009048,0.6999817,0.6958694
 uwind_nlon        = 512,
 uwind_nlat        = 256,
```
 
where the lines above correspond to the lower left coordinates of the field, the grid spacings, and the shape of the field.